In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 6 * 6, 7)  # 7 classes for FER2013 dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

# Define hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Define transformations to apply to the input images
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the FER2013 dataset
train_dataset = ImageFolder("data/train/", transform=transform)
test_dataset = ImageFolder("data/train/", transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the CNN model
model = CNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}")

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


Epoch [1/10], Step [100/449], Loss: 1.7896
Epoch [1/10], Step [200/449], Loss: 1.3462
Epoch [1/10], Step [300/449], Loss: 1.6751
Epoch [1/10], Step [400/449], Loss: 1.4387
Epoch [2/10], Step [100/449], Loss: 1.2965
Epoch [2/10], Step [200/449], Loss: 1.3867
Epoch [2/10], Step [300/449], Loss: 1.4126
Epoch [2/10], Step [400/449], Loss: 1.2889
Epoch [3/10], Step [100/449], Loss: 1.3117
Epoch [3/10], Step [200/449], Loss: 1.2372
Epoch [3/10], Step [300/449], Loss: 1.0476
Epoch [3/10], Step [400/449], Loss: 1.1642
Epoch [4/10], Step [100/449], Loss: 1.0936
Epoch [4/10], Step [200/449], Loss: 1.3203
Epoch [4/10], Step [300/449], Loss: 1.2951
Epoch [4/10], Step [400/449], Loss: 1.0954
Epoch [5/10], Step [100/449], Loss: 1.0158
Epoch [5/10], Step [200/449], Loss: 0.7565
Epoch [5/10], Step [300/449], Loss: 0.8185
Epoch [5/10], Step [400/449], Loss: 1.2496
Epoch [6/10], Step [100/449], Loss: 1.0516
Epoch [6/10], Step [200/449], Loss: 0.7893
Epoch [6/10], Step [300/449], Loss: 0.9101
Epoch [6/10